
# Wind Turbine Maintenance — **Final Explained Notebook (Step‑by‑Step)**

This notebook is a **fully explained**, **decision‑oriented** walkthrough of the pipeline that powers the one‑pager report.
For every code block, you'll see:
- **Purpose**: why the step exists and what question it answers
- **Inputs/Outputs**: what data goes in and what artifacts come out
- **Key parameters**: knobs to turn and why they matter
- **How to interpret**: what to look for in the outputs to make a decision

> Tip: Run cells in order (top → bottom). Artifacts are saved to `outputs_balanced_anomaly_nb_explained/`.



## 0) Environment Setup (optional)
**Purpose:** Ensure dependencies are available, especially in a new environment.  
**How to interpret:** If you see errors about missing packages later, come back and run this with the `!pip` line uncommented.


In [ ]:

# Optional: install dependencies if needed (uncomment if you get missing package errors)
# !pip install -U pandas numpy scikit-learn matplotlib xgboost lightgbm imbalanced-learn



## 1) Imports & Configuration
**Purpose:** Load the scientific Python stack and modeling libraries.  
**Key details:**
- We prefer **XGBoost → LightGBM → sklearn** for GBDT, depending on what's installed.
- **SMOTE** will balance the training data to fight class imbalance.
**How to interpret:** If you see warnings about XGBoost/LightGBM, we fall back gracefully to sklearn.


In [ ]:

import os, warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    average_precision_score, confusion_matrix, RocCurveDisplay, PrecisionRecallDisplay,
    precision_recall_curve
)
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.svm import OneClassSVM

# Prefer boosted trees backends in this order: XGBoost -> LightGBM -> sklearn
XGB_AVAILABLE = False
LGBM_AVAILABLE = False
SKLEARN_GB_AVAILABLE = True
try:
    from xgboost import XGBClassifier
    XGB_AVAILABLE = True
except Exception:
    warnings.warn("xgboost not available; will try lightgbm or sklearn GradientBoosting.")
try:
    from lightgbm import LGBMClassifier
    LGBM_AVAILABLE = True
except Exception:
    warnings.warn("lightgbm not available; may fall back to sklearn.")
try:
    from sklearn.ensemble import GradientBoostingClassifier
except Exception:
    SKLEARN_GB_AVAILABLE = False

# SMOTE for class imbalance
try:
    from imblearn.over_sampling import SMOTE
except Exception:
    raise SystemExit("Missing dependency: imbalanced-learn. Install with: pip install imbalanced-learn")



## 2) Load Dataset & Define Target
**Purpose:** Read the CSV and produce a clean **feature matrix (X)** and **binary target (y)**.  
**Inputs:** `wind_turbine_maintenance_data.csv` with a `Maintenance_Label` column.  
**Outputs:** 
- `X`: numeric predictors (sensor features, etc.)  
- `y`: 0 = normal, 1 = maintenance-needed (values > 0 become 1).  
**How to interpret:** Confirm label counts; strong imbalance is common and expected.


In [ ]:

DATA_PATH = os.getenv("WIND_DATA_PATH", "wind_turbine_maintenance_data.csv")
assert Path(DATA_PATH).exists(), f"CSV not found at: {DATA_PATH}"

df = pd.read_csv(DATA_PATH)
assert 'Maintenance_Label' in df.columns, "Expected 'Maintenance_Label' in dataset."

# Force binary target: maintenance if value > 0
y = pd.to_numeric(df['Maintenance_Label'], errors='coerce').fillna(0)
y = (y > 0).astype(int)

# Keep numeric features only for tree models
X = df.drop(columns=['Maintenance_Label']).select_dtypes(include=[np.number])

print("Label counts (dataset):\n", y.value_counts().to_string())
df.head(3)  # quick peek



## 3) Train/Test Split & **SMOTE** (training only)
**Purpose:** Create an unbiased evaluation and **balance** the training set to improve learning.  
**Key parameters:** `test_size=0.2`, `random_state=42` (reproducibility).  
**How to interpret:** SMOTE should roughly equalize class counts in `y_res`. We never oversample the **test** set.


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Label counts (train):", np.unique(y_train, return_counts=True))
print("Label counts (test) :", np.unique(y_test, return_counts=True))

smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)
print("Resampled label counts:", np.unique(y_res, return_counts=True))



## 4) Train Supervised Models (RF + GBDT)
**Purpose:** Learn to predict maintenance from historical signals.  
**Models:**
- **RandomForest** (robust, interpretable via importances)
- **GBDT** (XGBoost/LightGBM/sklearn fallback) often best on tabular data  
**How to interpret:** No printed metrics yet — that comes next.


In [ ]:

def make_rf():
    return RandomForestClassifier(n_estimators=400, n_jobs=-1, random_state=42)

def make_gbdt():
    if XGB_AVAILABLE:
        return XGBClassifier(n_estimators=600, max_depth=6, learning_rate=0.05,
                             subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
                             objective="binary:logistic", tree_method="hist",
                             random_state=42, n_jobs=-1, scale_pos_weight=1.0)
    if LGBM_AVAILABLE:
        return LGBMClassifier(n_estimators=700, learning_rate=0.05, subsample=0.8,
                              colsample_bytree=0.8, reg_lambda=1.0, objective="binary",
                              random_state=42, n_jobs=-1)
    return GradientBoostingClassifier(n_estimators=300, learning_rate=0.08, max_depth=3, random_state=42)

rf = make_rf().fit(X_res, y_res)
gb = make_gbdt().fit(X_res, y_res)
rf, gb



## 5) Evaluate: ROC/PR & **Precision–Recall vs Threshold**
**Purpose:** Understand ranking quality and choose an **operating threshold** for alerts.  
**Outputs:**  
- ROC/PR curves (model comparison)  
- PR vs Threshold (for each model) to **pick a cutoff** that meets precision/recall targets.  
**How to interpret:** Use PR‑vs‑threshold to set alert policies (e.g., *precision ≥ 0.5*).


In [ ]:

def get_proba(model, X):
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X)[:, 1]
    if hasattr(model, "decision_function"):
        d = model.decision_function(X)
        d_min, d_max = d.min(), d.max()
        return (d - d_min) / (d_max - d_min + 1e-9)
    return model.predict(X)

y_proba_rf = get_proba(rf, X_test)
y_proba_gb = get_proba(gb, X_test)

# ROC
fig, ax = plt.subplots(figsize=(5,4))
RocCurveDisplay.from_predictions(y_test, y_proba_rf, name="RandomForest", ax=ax)
RocCurveDisplay.from_predictions(y_test, y_proba_gb, name="GBDT", ax=ax)
ax.set_title("ROC — RF vs GBDT"); fig.tight_layout(); plt.show()

# PR
fig, ax = plt.subplots(figsize=(5,4))
PrecisionRecallDisplay.from_predictions(y_test, y_proba_rf, name="RandomForest", ax=ax)
PrecisionRecallDisplay.from_predictions(y_test, y_proba_gb, name="GBDT", ax=ax)
ax.set_title("Precision–Recall — RF vs GBDT"); fig.tight_layout(); plt.show()

# PR vs threshold
outdir_nb = Path("outputs_balanced_anomaly_nb_explained"); outdir_nb.mkdir(parents=True, exist_ok=True)
for name, probs in [("RandomForest", y_proba_rf), ("GBDT", y_proba_gb)]:
    p_vals, r_vals, thr_vals = precision_recall_curve(y_test, probs)
    thr_plot = np.concatenate([thr_vals, [thr_vals[-1] if thr_vals.size else 0.5]]) if thr_vals.size else np.array([0.5])
    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(thr_plot, p_vals, label="Precision")
    ax.plot(thr_plot, r_vals, label="Recall")
    ax.set_xlabel("Threshold"); ax.set_ylabel("Score"); ax.set_title(f"Precision–Recall vs Threshold — {name}")
    ax.legend(); fig.tight_layout()
    fig.savefig(outdir_nb / f"pr_vs_threshold_{name}.png", dpi=150)
    plt.show()



## 6) Threshold Sweeps + **Top Threshold Picks** (Supervised)
**Purpose:** Convert probabilities → decisions using a **threshold**, and summarize recommended operating points.  
**Outputs:**  
- `threshold_sweep_rf.csv`, `threshold_sweep_gbdt.csv`  
- `top_thresholds_supervised.csv` with: **best F1**, and best recall s.t. **precision ≥ 0.3** and **≥ 0.5**  
**How to interpret:** Choose the row that matches your false‑alarm tolerance and recall needs.


In [ ]:

def sup_thr_sweep(y_true, y_prob):
    rows = []
    thresholds = np.linspace(0.05, 0.95, 19)
    for thr in thresholds:
        y_pred = (y_prob >= thr).astype(int)
        rows.append({
            "threshold": thr,
            "accuracy": accuracy_score(y_true, y_pred),
            "precision": precision_score(y_true, y_pred, zero_division=0),
            "recall": recall_score(y_true, y_pred, zero_division=0),
            "f1": f1_score(y_true, y_pred, zero_division=0)
        })
    df_thr = pd.DataFrame(rows)
    best = df_thr.iloc[df_thr['f1'].values.argmax()]
    return float(best['threshold']), df_thr

def _top3_thresholds(df_thr, min_precisions=(0.3, 0.5)):
    out = {}
    if df_thr is None or df_thr.empty:
        return out
    i = df_thr['f1'].values.argmax()
    out['best_f1_threshold'] = float(df_thr.iloc[i]['threshold'])
    out['best_f1'] = float(df_thr.iloc[i]['f1'])
    out['best_f1_precision'] = float(df_thr.iloc[i]['precision'])
    out['best_f1_recall'] = float(df_thr.iloc[i]['recall'])
    for pmin in min_precisions:
        df_ok = df_thr[df_thr['precision'] >= pmin]
        if len(df_ok):
            j = df_ok['recall'].values.argmax()
            row = df_ok.iloc[j]
            out[f'prec>={pmin}_threshold'] = float(row['threshold'])
            out[f'prec>={pmin}_recall'] = float(row['recall'])
            out[f'prec>={pmin}_f1'] = float(row['f1'])
        else:
            out[f'prec>={pmin}_threshold'] = None
            out[f'prec>={pmin}_recall'] = None
            out[f'prec>={pmin}_f1'] = None
    return out

best_thr_rf, df_thr_rf = sup_thr_sweep(y_test, y_proba_rf)
best_thr_gb, df_thr_gb = sup_thr_sweep(y_test, y_proba_gb)

# Save sweeps + top picks
outdir_nb.mkdir(parents=True, exist_ok=True)
df_thr_rf.to_csv(outdir_nb / "threshold_sweep_rf.csv", index=False)
df_thr_gb.to_csv(outdir_nb / "threshold_sweep_gbdt.csv", index=False)

top_sup_df = pd.DataFrame([
    {'model':'RandomForest', **_top3_thresholds(df_thr_rf)},
    {'model':'GBDT', **_top3_thresholds(df_thr_gb)},
]).set_index('model')
top_sup_df.to_csv(outdir_nb / "top_thresholds_supervised.csv")
top_sup_df



## 7) **Unsupervised Anomaly Detection** (IsolationForest + One‑Class SVM) & PR‑vs‑Threshold
**Purpose:** Flag unusual behavior without labels; evaluate against labels **if available**.  
**Why train on normal only?** Anomalies are by definition rare and varied — learning the **normal manifold** is more stable.  
**Outputs:** ROC/PR, PR‑vs‑threshold plots for score cutoffs.
**How to interpret:** Use PR‑AUC and PR‑vs‑threshold to set **alerting policy**.


In [ ]:

# Train on *normal* training data only
X_train_norm = X_train[y_train == 0]

iso = IsolationForest(n_estimators=300, contamination=0.05, random_state=42).fit(X_train_norm)
oc  = OneClassSVM(kernel="rbf", gamma="scale", nu=0.05).fit(X_train_norm)

# Higher = more anomalous
iso_scores = -iso.score_samples(X_test)
oc_scores  = -oc.decision_function(X_test)

# ROC/PR
fig, ax = plt.subplots(figsize=(5,4))
RocCurveDisplay.from_predictions(y_test, iso_scores, name="IsolationForest", ax=ax)
RocCurveDisplay.from_predictions(y_test, oc_scores, name="OneClassSVM", ax=ax)
ax.set_title("Anomaly ROC — IF vs OCSVM"); fig.tight_layout(); plt.show()

fig, ax = plt.subplots(figsize=(5,4))
PrecisionRecallDisplay.from_predictions(y_test, iso_scores, name="IsolationForest", ax=ax)
PrecisionRecallDisplay.from_predictions(y_test, oc_scores, name="OneClassSVM", ax=ax)
ax.set_title("Anomaly PR — IF vs OCSVM"); fig.tight_layout(); plt.show()

# PR vs threshold (anomaly scores)
for name, scores in [("IsolationForest", iso_scores), ("OneClassSVM", oc_scores)]:
    p_vals, r_vals, thr_vals = precision_recall_curve(y_test, scores)
    thr_plot = np.concatenate([thr_vals, [thr_vals[-1] if thr_vals.size else scores.mean()]]) if thr_vals.size else np.array([scores.mean()])
    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(thr_plot, p_vals, label="Precision")
    ax.plot(thr_plot, r_vals, label="Recall")
    ax.set_xlabel("Score threshold"); ax.set_ylabel("Score"); ax.set_title(f"PR vs Threshold — {name}")
    ax.legend(); fig.tight_layout()
    fig.savefig(outdir_nb / f"anomaly_pr_vs_threshold_{name}.png", dpi=150)
    plt.show()



## 8) Threshold Sweeps + **Top Threshold Picks** (Anomaly)
**Purpose:** Convert anomaly scores → decisions using a score cutoff; summarize recommended cutoffs.  
**Outputs:**  
- `threshold_sweep_iso.csv`, `threshold_sweep_ocsvm.csv`  
- `top_thresholds_anomaly.csv` with **best F1** and precision‑constrained picks  
**How to interpret:** Lower cutoffs increase recall (more alerts); higher cutoffs increase precision (fewer alerts).


In [ ]:

def threshold_sweep_scores(y_true, scores):
    rows = []
    thr_values = np.linspace(np.percentile(scores, 5), np.percentile(scores, 95), 31)
    for thr in thr_values:
        y_pred = (scores >= thr).astype(int)
        rows.append({
            "threshold": thr,
            "accuracy": accuracy_score(y_true, y_pred),
            "precision": precision_score(y_true, y_pred, zero_division=0),
            "recall": recall_score(y_true, y_pred, zero_division=0),
            "f1": f1_score(y_true, y_pred, zero_division=0)
        })
    df_thr = pd.DataFrame(rows)
    best = df_thr.iloc[df_thr['f1'].values.argmax()]
    return float(best['threshold']), df_thr

best_thr_iso, df_thr_iso = threshold_sweep_scores(y_test, iso_scores)
best_thr_oc,  df_thr_oc  = threshold_sweep_scores(y_test, oc_scores)

df_thr_iso.to_csv(outdir_nb / "threshold_sweep_iso.csv", index=False)
df_thr_oc.to_csv(outdir_nb / "threshold_sweep_ocsvm.csv", index=False)

def _top3_thresholds(df_thr, min_precisions=(0.3, 0.5)):
    out = {}
    if df_thr is None or df_thr.empty:
        return out
    i = df_thr['f1'].values.argmax()
    out['best_f1_threshold'] = float(df_thr.iloc[i]['threshold'])
    out['best_f1'] = float(df_thr.iloc[i]['f1'])
    out['best_f1_precision'] = float(df_thr.iloc[i]['precision'])
    out['best_f1_recall'] = float(df_thr.iloc[i]['recall'])
    for pmin in min_precisions:
        df_ok = df_thr[df_thr['precision'] >= pmin]
        if len(df_ok):
            j = df_ok['recall'].values.argmax()
            row = df_ok.iloc[j]
            out[f'prec>={pmin}_threshold'] = float(row['threshold'])
            out[f'prec>={pmin}_recall'] = float(row['recall'])
            out[f'prec>={pmin}_f1'] = float(row['f1'])
        else:
            out[f'prec>={pmin}_threshold'] = None
            out[f'prec>={pmin}_recall'] = None
            out[f'prec>={pmin}_f1'] = None
    return out

top_anom_df = pd.DataFrame([
    {'model':'IsolationForest', **_top3_thresholds(df_thr_iso)},
    {'model':'OneClassSVM', **_top3_thresholds(df_thr_oc)},
]).set_index('model')
top_anom_df.to_csv(outdir_nb / "top_thresholds_anomaly.csv")
top_anom_df



## 9) Export Per‑Row Scores & Top Anomalies
**Purpose:** Produce tables for inspection tickets and dashboards.  
**Outputs:**  
- `anomaly_scores_test.csv` — per row scores and true label  
- `top_anomalies_*.csv` — top 50 highest‑scored anomalies per method  
**How to interpret:** Start with the highest scores; cross‑check **Turbine_ID** if present.


In [ ]:

# Predictions at best thresholds (for reference)
y_pred_iso = (iso_scores >= best_thr_iso).astype(int)
y_pred_oc  = (oc_scores  >= best_thr_oc).astype(int)

# Combined scores
combined_scores = pd.DataFrame({
    "row_index": X_test.index,
    "true_label": y_test.values,
    "iso_score": iso_scores,
    "ocsvm_score": oc_scores
}).sort_values("iso_score", ascending=False)

if "Turbine_ID" in df.columns:
    combined_scores["Turbine_ID"] = df.loc[X_test.index, "Turbine_ID"]

combined_scores.to_csv(outdir_nb / "anomaly_scores_test.csv", index=False)

# Top anomalies per method
for name, scores in [("IsolationForest", iso_scores), ("OneClassSVM", oc_scores)]:
    top_idx = np.argsort(scores)[::-1][:50]
    top_df = pd.DataFrame({
        "row_index": X_test.index[top_idx],
        "score": scores[top_idx],
        "true_label": y_test.iloc[top_idx].values
    })
    if "Turbine_ID" in df.columns:
        top_df["Turbine_ID"] = df.loc[X_test.index[top_idx], "Turbine_ID"].values
    top_df.to_csv(outdir_nb / f"top_anomalies_{name}.csv", index=False)

combined_scores.head(3)



## 10) Feature Importances (What Signals Drive Decisions?)
**Purpose:** Identify the most influential sensors/features driving predictions.  
**How to interpret:** Features that appear as **top‑ranked** across both RF and GBDT deserve operational focus (QA, spare parts, tighter alarms).


In [ ]:

def plot_importance(model, feature_names, title, outname):
    if not hasattr(model, "feature_importances_"):
        print(f"No feature_importances_ for {title}."); return
    imp = model.feature_importances_
    order = np.argsort(imp)[::-1][:15]
    fig, ax = plt.subplots(figsize=(7,5))
    ax.barh(range(len(order))[::-1], imp[order][::-1])
    ax.set_yticks(range(len(order))[::-1])
    ax.set_yticklabels([feature_names[i] for i in order][::-1])
    ax.set_xlabel("Importance"); ax.set_title(title)
    fig.tight_layout(); fig.savefig(outdir_nb / outname, dpi=150); plt.show()

feature_names = X.columns.tolist()
plot_importance(rf, feature_names, "Feature Importance — RandomForest (SMOTE)", "feature_importance_rf_smote.png")
plot_importance(gb, feature_names, "Feature Importance — GBDT (SMOTE)", "feature_importance_gbdt_smote.png")



## 11) Next Steps (How to Use These Results)
- Use **PR‑vs‑threshold** and **Top threshold picks** to set alert policies (e.g., *precision ≥ 0.5* or *maximize F1*).
- Track **precision/recall** weekly after deployment and adjust thresholds as capacity and costs evolve.
- Add **temporal features** (rolling means, rates of change) to capture trends for better separability.
- Consider **SHAP** for per‑prediction explanations when presenting to operators.
